# Importation des librairies

In [1]:
! pip install -q pandas-Profiling

In [33]:
import requests
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import shape
from io import StringIO
from sklearn.impute import KNNImputer

# Importation des bases

## Import des bases de données via API de l'OpenDataSoft

L'éxécution de la requête API prend assez de temps

In [2]:
def get_data_ODS(dataset_name):
    """
    Fonction qui permet de récupérer des données via l'API d'OpenDataSoft
    Elle retourne un dataframe
    
    """
    df = pd.DataFrame()
    # URL de base pour accéder à l'API d'ODS
    base_url_ODS = "https://odre.opendatasoft.com/api/explore/v2.1"
    
    dataset_path = f"/catalog/datasets/{dataset_name}/exports/json?lang=fr&timezone=Europe%2FBerlin"
    url = f"{base_url_ODS}{dataset_path}"
    
    # Exécution de la requête GET
    
    response = requests.get(url)
    # Vérification si la requête a réussi
    if response.status_code == 200:
        # Extraction des données
        data = response.json()
        # Conversion des résultats en DataFrame
        df = pd.DataFrame(data)
    else:
        print(f"Erreur lors de la requête: {response.status_code}")

    return df


def get_data_ODSW(dataset_name, output = "json"):
    """
    Fonction qui permet de récupérer des données via l'API  publique d'OpenDataSoft
    Elle retourne un dataframe
    
    """
    df = pd.DataFrame()
    # URL de base pour accéder à l'API d'ODS
    base_url_ODS = "https://public.opendatasoft.com/api/explore/v2.1"
    
    dataset_path = f"/catalog/datasets/{dataset_name}/exports/{output}?lang=fr&timezone=Europe%2FBerlin"
    url = f"{base_url_ODS}{dataset_path}"
    
    # Exécution de la requête GET
    response = requests.get(url)
    # Vérification si la requête a réussi
    if response.status_code == 200:
        # Extraction des données
        if output == "json":
            data = response.json()
            # Conversion des résultats en DataFrame
            df = pd.DataFrame(data)
        else:
            data = response.json()
            geometries = [shape(feature['geometry']) for feature in data['features']]
            properties = [feature['properties'] for feature in data['features']]
    
            # Créer un GeoDataFrame en combinant les géométries et les propriétés
            df = gpd.GeoDataFrame(geometry=geometries, data=properties)
    else:
        print(f"Erreur lors de la requête: {response.status_code}")

    return df

In [4]:
consumption_dataset_name = "consommation-quotidienne-brute-regionale"
weather_dataset_name = "donnees-synop-essentielles-omm"
geographic_dataset_name = "georef-france-commune"
holiday_dataset_name = "jours-ouvres-week-end-feries-france-2010-a-2030"
data_consumption = get_data_ODS(consumption_dataset_name)
data_weather = get_data_ODSW(weather_dataset_name)
data_geo = get_data_ODSW(geographic_dataset_name, output="geojson")
holiday_dataset = get_data_ODSW(holiday_dataset_name)

On traite les colonnes du dataset geographique pour enregistrer les données sous format GeoJson

In [31]:
#On recupère les colonnes importantes de la base de données
geo_cols = ['geo_point_2d', 'year', 'reg_code', 'reg_name', 'dep_code', 'dep_name','geometry']
data_geo = data_geo[geo_cols]
treated_geo_cols = ['year', 'reg_code', 'reg_name', 'dep_code', 'dep_name']
#On effectue un nettoyage des données contenues dans les colonnes
data_geo[treated_geo_cols] = data_geo[treated_geo_cols].applymap(lambda x: x.replace('[', '').replace(']', '') if isinstance(x, str) else x)
data_geo[treated_geo_cols] = data_geo[treated_geo_cols].applymap(lambda x: ''.join(filter(lambda char: char.isalnum() or char in ['_', '-', '@', '#'], str(x))))

,geo_point_2d,year,reg_code,reg_name,dep_code,dep_name,geometry
0,"{'lon': 4.186473859960533, 'lat': 48.520816933...",2023,44,GrandEst,10,Aube,"POLYGON ((4.18730 48.54392, 4.18724 48.54250, ..."
1,"{'lon': 4.598596436015939, 'lat': 48.306537350...",2023,44,GrandEst,10,Aube,"POLYGON ((4.62567 48.32423, 4.62429 48.32356, ..."
2,"{'lon': 4.6153344590215, 'lat': 48.44294278327...",2023,44,GrandEst,10,Aube,"POLYGON ((4.63562 48.45660, 4.63535 48.45667, ..."
3,"{'lon': 3.8349133102621433, 'lat': 48.10864367...",2023,44,GrandEst,10,Aube,"POLYGON ((3.80166 48.10673, 3.80124 48.10597, ..."
4,"{'lon': 1.8228167609346886, 'lat': 43.33065948...",2023,76,Occitanie,11,Aude,"POLYGON ((1.83985 43.32973, 1.83966 43.32982, ..."


## Import d'une base de données complémentaire (celle des régions, communes, et départements de France

Cette importation va se faire à travers une requête via l'API de Datagouv

In [3]:
#Identifiant du dataset
dataset_id = "4a7c837bb6da8e363604082bcc8b2e504cf08038"
dataset_sha1 = "dbe8a621-a9c4-4bc3-9cae-be1699c5ff25"

# URL de base pour accéder à l'API
base_url = "https://www.data.gouv.fr/api/1/"

# Chemin pour accéder aux enregistrements du dataset
dataset_path = f"datasets/r/{dataset_sha1}?dataset={dataset_id}"

# Construction de l'URL complète
url = f"{base_url}{dataset_path}"

# Exécution de la requête GET
response = requests.get(url)

# Vérification si la requête a réussi
if response.status_code == 200:
    # Lecture du contenu CSV
    data_communes = pd.read_csv(StringIO(response.content.decode('utf-8')))
else:
    print(f"Erreur lors de la requête: {response.status_code}")

On enregistre la base pour ne plus avoir à éxécuter la requête API à chaque étape

In [6]:
data_consumption.to_csv('data_consumption.csv', index=False)
data_communes.to_csv('data_communes.csv', index=False)
data_weather.to_csv('data_weather.csv', index = False)
data_geo.to_file("data_geo.geojson", driver='GeoJSON')

In [4]:
data_consumption = pd.read_csv('data_consumption.csv')
data_communes = pd.read_csv('data_communes.csv')
data_weather = pd.read_csv('data_weather.csv')
data_geo = gpd.read_file("data_geo.geojson")

ERROR 1: PROJ: proj_create_from_database: Open of /opt/mamba/share/proj failed


Les dataframes de consommation et météorologiques  étant sous forme de séries temporelles, on place les dates en index.

In [5]:
data_consumption['date'] = pd.to_datetime(data_consumption['date'])
data_consumption.set_index('date', inplace=True)

In [6]:
data_consumption.head()

,date_heure,heure,code_insee_region,region,consommation_brute_gaz_grtgaz,statut_grtgaz,consommation_brute_gaz_terega,statut_terega,consommation_brute_gaz_totale,consommation_brute_electricite_rte,statut_rte,consommation_brute_totale
date,,,,,,,,,,,,
2019-02-01,2019-02-01T06:00:00+01:00,06:00,27,Bourgogne-Franche-Comté,5053.0,Définitif,NaN,NaN,5053.0,3052,Définitif,8105.0
2019-02-01,2019-02-01T07:00:00+01:00,07:00,24,Centre-Val de Loire,4079.0,Définitif,NaN,NaN,4079.0,3117,Définitif,7196.0
2019-02-01,2019-02-01T07:00:00+01:00,07:00,52,Pays de la Loire,5795.0,Définitif,NaN,NaN,5795.0,4646,Définitif,10441.0
2019-02-01,2019-02-01T07:30:00+01:00,07:30,24,Centre-Val de Loire,NaN,NaN,NaN,NaN,NaN,3252,Définitif,NaN
2019-02-01,2019-02-01T07:30:00+01:00,07:30,28,Normandie,NaN,NaN,NaN,NaN,NaN,4911,Définitif,NaN


Choix de l'année

In [7]:
annee_choisie = 2022
data_consumption = data_consumption[data_consumption.index.year == annee_choisie]


In [8]:
data_communes.isna().sum()

code_commune_INSEE          0
nom_commune_postal          0
code_postal                 0
libelle_acheminement        0
ligne_5                 35944
latitude                  269
longitude                 269
code_commune                7
article                 36621
nom_commune                 0
nom_commune_complet         0
code_departement            7
nom_departement           267
code_region               267
nom_region                267
dtype: int64

In [9]:
col_del = pd.Index(['date_heure', 'consommation_brute_gaz_grtgaz', 'statut_grtgaz', 'consommation_brute_gaz_terega', 'statut_terega','consommation_brute_gaz_totale','statut_rte', 'consommation_brute_totale'])
data_consumption.drop(col_del, axis=1,inplace=True)
data_consumption.isna().sum()

heure                                 0
code_insee_region                     0
region                                0
consommation_brute_electricite_rte    0
dtype: int64

## Aggrégation des consommations par date et par région

In [10]:
data_consumption1 = data_consumption.groupby(['code_insee_region','date'], as_index = False)
data_consumption2 = data_consumption.groupby(['code_insee_region','date','heure'], as_index = False)

In [11]:
data_consumption1 = data_consumption1['consommation_brute_electricite_rte'].sum()
data_consumption2 = data_consumption2['consommation_brute_electricite_rte'].sum()

In [12]:
data_consumption1.head()

,code_insee_region,consommation_brute_electricite_rte
0,11,345462
1,11,356965
2,11,414973
3,11,429347
4,11,484779


### Calcul des superficies par département

### Calcul de la superficie

In [13]:
# On divise par par 1e6 pour obtenir la superficie par commune en km²
data_geo['superficie_comm'] = data_geo['geometry'].area / 1e6  

#On fait un regroupement par département, et on calcule la superficie par département en sommant les autres superficies
data_geo['superficie_dep'] = data_geo.groupby('dep_code')['superficie_comm'].transform('sum')

/tmp/ipykernel_30800/3138837871.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  data_geo['superficie_comm'] = data_geo['geometry'].area / 1e6


### Traitement de la base weather

In [14]:
nouveaux_noms_colonnes={'numer_sta':'num_station', 'date':'date_UTC', 'pmer':'pression_mer', 'tend':'variation_pression_3h', 'cod_tend':'type_tendance_baro', 'dd':'direction_vent', 'ff':'vitesse_vent', 't':'temperature', 'td':'point_de_rosee',
       'u':'humidite', 'vv':'visibilite_horizontale', 'ww':'temps_present', 'w1':'temps_passe_1', 'w2':'temps_passe_2', 'n':'nebulosite_totale', 'nbas':'nebulosite_nuage_etage_inf', 'hbas':'hauteur_base_nuage_etage_inf', 'cl':'type_nuage_etage_inf', 'cm':'type_nuage_etage_moyen', 'ch':'type_nuage_etage_sup',
       'pres':'pression_station', 'niv_bar':'niveau_barometriq', 'geop':'geopotentiel', 'tend24':'variation_pression_24h', 'tn12':'temperature_min_sur_12h', 'tn24':'temperature_min_sur_24h', 'tx12':'temperature_max_sur_12h', 'tx24':'temperature_max_sur_24h',
       'tminsol':'temperature_min_sol_sur_12h', 'sw':'methode_mesure_temperature_thermometre_mouille', 'tw':'temperature_thermometre_mouille', 'raf10':'rafales_10_dernieres_minutes', 'rafper':'rafales_sur_une_periode', 'per':'periode_mesure_rafales', 'etat_sol':'etat_du_sol', 'ht_neige':'hauteur_totale_couche_neige/glace_au_sol',
       'ssfrai':'hauteur_neige_fraiche', 'perssfrai':'periode_mesure_neige_fraiche', 'rr1':'precipitation_dans_1_derniere_heure', 'rr3':'precipitation_dans_3_derniere_heure', 'rr6':'precipitation_dans_6_derniere_heure', 'rr12':'precipitation_dans_12_derniere_heure', 'rr24':'precipitation_dans_24_derniere_heure', 'phenspe1':'phenomene_special_1',
       'phenspe2':'phenomene_special_2', 'phenspe3':'phenomene_special_3', 'phenspe4':'phenomene_special_4', 'nnuage1':'nebulosite_couche_nuageuse_1', 'ctype1':'type_de_nuage_1', 'hnuage1':'hauteur_de_base_nuage_1',
       'nnuage2':'nebulosite_couche_nuageuse_2', 'ctype2':'type_de_nuage_2', 'hnuage2':'hauteur_de_base_nuage_2', 'nnuage3':'nebulosite_couche_nuageuse_3', 'ctype3':'type_de_nuage_3', 'hnuage3':'hauteur_de_base_nuage_3',
       'nnuage4':'nebulosite_couche_nuageuse_4', 'ctype4':'type_de_nuage_4', 'hnuage4':'hauteur_de_base_nuage_4', 'coordonnees':'coordonnees', 'nom':'nom_commune',
       'type_de_tendance_barometrique':'type_de_tendance_barometrique', 'temps_passe_1':'temps_passe_1', 'temps_present':'temps_present', 'tc':'temperature_degre_celcius',
       'tn12c':'temperature_celcius_min_sur_12h', 'tn24c':'temperature_celcius_min_sur_24h', 'tx12c':'temperature_celcius_max_sur_12h', 'tx24c':'temperature_celcius_max_sur_24h', 'tminsolc':'temperature_celcius_min_sol_sur_12h', 'latitude':'latitude', 'longitude':'longitude',
       'altitude':'altitude', 'libgeo':'libelle_geolocalisation', 'codegeo':'code_geolocalisation', 'nom_epci':'nom_EPCI', 'code_epci':'code_EPCI', 'nom_dept':'nom_departement',
       'code_dep':'code_departement', 'nom_reg':'nom_region', 'code_reg':'code_region', 'mois_de_l_annee':'mois'}

In [15]:
base_meteo_modifie_1 = data_weather.rename(columns=nouveaux_noms_colonnes)

## Identification des variables météoroliques à retenir

Il est question d'identifier les les variables météorologiques se reférant aux espects retenus Après une revue de la littérature:
* __Température :__ les variables identifiées sont la température en degré celcius ('temperature_degre_celcius'), la température minimale sur les 24 dernières heures ('temperature_celcius_min_sur_24h'), et la température maximale sur les 24 dernières heures ('temperature_celcius_max_sur_24h')
* __Vitesse du vent :__ la variable identifiée est la vitesse du vent ('vitesse_vent')
* __Couverture nuageuse :__ la variable identifié est la nébulosité totale et renvoie à la quantité totale de nuages couvrant le ciel ou plus simplement le degré de couverture nuageuse dans le ciel ('nebulosite_totale')
* __Humidité :__ la variable identifiée est l'indice d'humidité ('humidite')
* __Rayonnement global :__ la variable identifiée pour capter cet aspect est la distance maximale à laquelle des objets peuvent être clairement discernés à l'horizon ('visibilite_horizontale')

In [16]:
base_meteo_modifie_1.columns

Index(['num_station', 'date_UTC', 'pression_mer', 'variation_pression_3h',
       'type_tendance_baro', 'direction_vent', 'vitesse_vent', 'temperature',
       'point_de_rosee', 'humidite', 'visibilite_horizontale', 'temps_present',
       'temps_passe_1', 'temps_passe_2', 'nebulosite_totale',
       'nebulosite_nuage_etage_inf', 'hauteur_base_nuage_etage_inf',
       'type_nuage_etage_inf', 'type_nuage_etage_moyen',
       'type_nuage_etage_sup', 'pression_station', 'niveau_barometriq',
       'geopotentiel', 'variation_pression_24h', 'temperature_min_sur_12h',
       'temperature_min_sur_24h', 'temperature_max_sur_12h',
       'temperature_max_sur_24h', 'temperature_min_sol_sur_12h',
       'methode_mesure_temperature_thermometre_mouille',
       'temperature_thermometre_mouille', 'rafales_10_dernieres_minutes',
       'rafales_sur_une_periode', 'periode_mesure_rafales', 'etat_du_sol',
       'hauteur_totale_couche_neige/glace_au_sol', 'hauteur_neige_fraiche',
       'periode_mesure_

In [18]:
weather_vars = ['num_station', 'date_UTC', 'vitesse_vent', 'temperature','nebulosite_totale','temperature_degre_celcius', 'temperature_celcius_min_sur_24h', 'temperature_celcius_max_sur_24h','latitude', 'longitude','altitude', 'libelle_geolocalisation','nom_departement', 'code_departement','nom_region', 'code_region', 'mois', 'visibilite_horizontale', 'coordonnees','nom_commune',]
base_meteo_modifie_1 = base_meteo_modifie_1[weather_vars]

### Vérification et gestion des valeurs manquantes pour les variables d'agrégation

#### Vérification des régions et départements manquants

In [19]:
print(base_meteo_modifie_1['nom_region'].isnull().sum())
print(base_meteo_modifie_1['code_region'].isnull().sum())
print(base_meteo_modifie_1['nom_departement'].isnull().sum())
print(base_meteo_modifie_1['code_departement'].isnull().sum())
print(base_meteo_modifie_1['nom_commune'].isnull().sum())

242657
242657
242657
242657
0


On constate que tous les noms de commune sont renseignés, mais certains noms de région et département ne le sont pas. Nous allons donc utiliser les noms de commune pour renseigner les départements et régions manquants.

Nous explorons la possibilité de completer les régions et départements manquants à partir de la base de données des communes, régions et départements.

In [20]:
test=pd.merge(base_meteo_modifie_1, data_communes, on=['nom_commune'], how='left')
print(test['nom_region_y'].isnull().sum())
print(test['code_region_y'].isnull().sum())
print(test['nom_departement_y'].isnull().sum())
print(test['code_departement_y'].isnull().sum())
print(test['nom_commune'].isnull().sum())

2353668
2353668
2353668
2353668
0


## Agrégation des variables météorologiques

In [21]:
base_meteo_modifie_2=base_meteo_modifie_1

Il est ici question de construire les variables qui seront utilisées dans les analyses. Les données seront agrégées au niveau régional et à une fréquence journalière.

Il est ici question de construire les variables qui seront utilisées dans les analyses. Les données seront agrégées au niveau régional et à une fréquence journalière.

In [22]:
def extraire_sous_chaine(chaine, debut, fin):
    """
    Extrait la sous-chaîne de 'debut' à 'fin' (inclus) de la chaîne donnée.
    """
    if debut < 0 or fin >= len(chaine):
        raise ValueError("Indices de début ou de fin invalides.")
    return chaine[debut : fin + 1]

Nous appliquons la fonction à la base:

In [23]:
base_meteo_modifie_2['jour']=base_meteo_modifie_2['date_UTC'].apply(extraire_sous_chaine,debut=0,fin=9)
base_meteo_modifie_2['heure']=base_meteo_modifie_2['date_UTC'].apply(extraire_sous_chaine,debut=11,fin=18)

In [24]:
base_meteo_modifie_2['jour'] = pd.to_datetime(base_meteo_modifie_2['jour'])
base_meteo_modifie_2.set_index('jour', inplace=True)
base_meteo_modifie_2 = base_meteo_modifie_2[base_meteo_modifie_2.index.year == annee_choisie]

## Agrégation des variables à l'échelle départementale

Tout d'abord, agrégeons les variables 'temperature_degre_celcius','vitesse_vent', 'nebulosite_totale', 'humidite', 'visibilite_horizontale'

In [25]:
agregation_dep_mean = base_meteo_modifie_2.groupby(['jour','code_departement']).agg({'temperature_degre_celcius': 'mean', 'vitesse_vent': 'mean','nebulosite_totale':'mean','visibilite_horizontale':'mean'})
agregation_dep_mean = agregation_dep_mean.rename(columns={'jour':'jour','code_departement':'code_departement','temperature_degre_celcius':'temperature_degre_celcius_departement','vitesse_vent':'vitesse_vent_departement','nebulosite_totale':'nebulosite_totale_departement','visibilite_horizontale':'visibilite_horizontale_departement'})

In [26]:
base_meteo_modifie_3 = pd.merge(base_meteo_modifie_2, agregation_dep_mean, on=['jour','code_departement'], how='left')

In [27]:
data_geo.rename(columns={'dep_code': 'code_departement'}, inplace=True)
data_geo['code_departement'] = data_geo['code_departement'].astype(str)
base_meteo_modifie_4 = data_geo.merge(base_meteo_modifie_3, on='code_departement')
#base_meteo_modifie_4 = gpd.GeoDataFrame(pd.merge(base_meteo_modifie_3, data_geo, on=['code_departement'], how='left'),  geometry='geometry')

Index(['geo_point_2d', 'year', 'reg_code', 'reg_name', 'code_departement',
       'dep_name', 'geometry', 'superficie_comm', 'superficie_dep',
       'num_station', 'date_UTC', 'vitesse_vent', 'temperature',
       'nebulosite_totale', 'temperature_degre_celcius',
       'temperature_celcius_min_sur_24h', 'temperature_celcius_max_sur_24h',
       'latitude', 'longitude', 'altitude', 'libelle_geolocalisation',
       'nom_departement', 'nom_region', 'code_region', 'mois',
       'visibilite_horizontale', 'coordonnees', 'nom_commune', 'heure',
       'temperature_degre_celcius_departement', 'vitesse_vent_departement',
       'nebulosite_totale_departement', 'visibilite_horizontale_departement'],
      dtype='object')

### Aggrégation par région en appliquant les pondérations que sont les superficies des départements

Créons un dictionnaire pour stocker les critères d'agrégation

In [30]:
criteres_agregation = {'temperature_degre_celcius_departement': lambda x: np.average(x, weights=base_meteo_modifie_4.loc[x.index, 'superficie_dep']),'vitesse_vent_departement': lambda x: np.average(x, weights=base_meteo_modifie_4.loc[x.index, 'superficie_dep']),'nebulosite_totale_departement': lambda x: np.average(x, weights=base_meteo_modifie_4.loc[x.index, 'superficie_dep']),'visibilite_horizontale_departement': lambda x: np.average(x, weights=base_meteo_modifie_4.loc[x.index, 'superficie_dep'])}

Appliquons l'agrégation groupée en utilisant les critères spécifiés

In [ ]:
agregation_reg_mean = base_meteo_modifie_4.groupby([base_meteo_modifie_4.index,'code_region']).agg(criteres_agregation)
agregation_reg_mean = agregation_reg_mean.rename(columns={'code_region':'code_region','temperature_degre_celcius_departement':'temperature_degre_celcius_region','vitesse_vent_departement':'vitesse_vent_region','nebulosite_totale_departement':'nebulosite_totale_region','visibilite_horizontale_departement':'visibilite_horizontale_region'})

Fusion de la base agrégée obtenue et de la base de départ dans une nouvelle base

In [ ]:
base_meteo_modifie_5 = pd.merge(base_meteo_modifie_4, agregation_reg_mean, on=['jour','code_region'], how='left')

Aperçu de la base obtenue

In [ ]:
base_meteo_modifie_5.head()

## Agrégation des autres variables (températures minimales et maximales)

Imputation des variables manquantes : KNN Imputer

On se dit que les zones ayant des données semblables météorologiques "se ressemblent"

On vérifie d'abord que toutes les variables sont numériques; 

In [ ]:
# Cas des données de consommation
data_consumption.dtypes

# Cas des données des communes
data_communes.dtypes


On convertit les colonnes

In [ ]:
cols = []
for col in cols:
    df[col] = df[col].astype(float)


On applique ensuite le KNN imputer

In [ ]:

# Initialiser l'imputeur KNN
cons_imputer = KNNImputer(n_neighbors=2)
com_imputer = KNNImputer(n_neighbors=2)

# Appliquer l'imputation
data_consumption_imputed = pd.DataFrame(cons_imputer.fit_transform(data_consumption), columns=data_consumption.columns)
data_communes_imputed = pd.DataFrame(com_imputer.fit_transform(data_communes), columns = data_communes.columns)


### Traitement de la base geographique pour le calcul des surfaces